In [57]:
#!pip install docker

In [58]:
#!pip install clickhouse_driver

In [59]:
#!docker run -d --name clickhouse-server --ulimit nofile=262144:262144 -p 9000:9000 yandex/clickhouse-server

In [60]:
from clickhouse_driver import Client

client = Client(host='localhost',  # Use 'localhost' or '127.0.0.1' for a local server
                user='default',    # Default user, adjust if you've changed the user
                password='',       # Default installation has no password for 'default' user
                port=9000)         # Default TCP port for ClickHouse

# Attempt to execute a query
try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Encountered an error: {e}")

[('trips',)]


In [61]:
client.execute('SHOW TABLES')

[('trips',)]

In [62]:
client.execute(
'''
CREATE TABLE trips_2
(
    `trip_id` UInt32,
    `vendor_id` Enum8('1' = 1, '2' = 2, '3' = 3, '4' = 4, 'CMT' = 5, 'VTS' = 6, 'DDS' = 7, 'B02512' = 10, 'B02598' = 11, 'B02617' = 12, 'B02682' = 13, 'B02764' = 14, '' = 15),
    `pickup_date` Date,
    `pickup_datetime` DateTime,
    `dropoff_date` Date,
    `dropoff_datetime` DateTime,
    `store_and_fwd_flag` UInt8,
    `rate_code_id` UInt8,
    `pickup_longitude` Float64,
    `pickup_latitude` Float64,
    `dropoff_longitude` Float64,
    `dropoff_latitude` Float64,
    `passenger_count` UInt8,
    `trip_distance` Float64,
    `fare_amount` Float32,
    `extra` Float32,
    `mta_tax` Float32,
    `tip_amount` Float32,
    `tolls_amount` Float32,
    `ehail_fee` Float32,
    `improvement_surcharge` Float32,
    `total_amount` Float32,
    `payment_type` Enum8('UNK' = 0, 'CSH' = 1, 'CRE' = 2, 'NOC' = 3, 'DIS' = 4),
    `trip_type` UInt8,
    `pickup` FixedString(25),
    `dropoff` FixedString(25),
    `cab_type` Enum8('yellow' = 1, 'green' = 2, 'uber' = 3),
    `pickup_nyct2010_gid` Int8,
    `pickup_ctlabel` Float32,
    `pickup_borocode` Int8,
    `pickup_ct2010` String,
    `pickup_boroct2010` String,
    `pickup_cdeligibil` String,
    `pickup_ntacode` FixedString(4),
    `pickup_ntaname` String,
    `pickup_puma` UInt16,
    `dropoff_nyct2010_gid` UInt8,
    `dropoff_ctlabel` Float32,
    `dropoff_borocode` UInt8,
    `dropoff_ct2010` String,
    `dropoff_boroct2010` String,
    `dropoff_cdeligibil` String,
    `dropoff_ntacode` FixedString(4),
    `dropoff_ntaname` String,
    `dropoff_puma` UInt16
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(pickup_date)
ORDER BY pickup_datetime;
'''
)

[]

In [63]:
client.execute('SHOW TABLES')

[('trips',), ('trips_2',)]

In [64]:
client.execute(
'''
INSERT INTO trips
SELECT * FROM s3(
    'https://datasets-documentation.s3.eu-west-3.amazonaws.com/nyc-taxi/trips_{1..2}.gz',
    'TabSeparatedWithNames', "
    `trip_id` UInt32,
    `vendor_id` Enum8('1' = 1, '2' = 2, '3' = 3, '4' = 4, 'CMT' = 5, 'VTS' = 6, 'DDS' = 7, 'B02512' = 10, 'B02598' = 11, 'B02617' = 12, 'B02682' = 13, 'B02764' = 14, '' = 15),
    `pickup_date` Date,
    `pickup_datetime` DateTime,
    `dropoff_date` Date,
    `dropoff_datetime` DateTime,
    `store_and_fwd_flag` UInt8,
    `rate_code_id` UInt8,
    `pickup_longitude` Float64,
    `pickup_latitude` Float64,
    `dropoff_longitude` Float64,
    `dropoff_latitude` Float64,
    `passenger_count` UInt8,
    `trip_distance` Float64,
    `fare_amount` Float32,
    `extra` Float32,
    `mta_tax` Float32,
    `tip_amount` Float32,
    `tolls_amount` Float32,
    `ehail_fee` Float32,
    `improvement_surcharge` Float32,
    `total_amount` Float32,
    `payment_type` Enum8('UNK' = 0, 'CSH' = 1, 'CRE' = 2, 'NOC' = 3, 'DIS' = 4),
    `trip_type` UInt8,
    `pickup` FixedString(25),
    `dropoff` FixedString(25),
    `cab_type` Enum8('yellow' = 1, 'green' = 2, 'uber' = 3),
    `pickup_nyct2010_gid` Int8,
    `pickup_ctlabel` Float32,
    `pickup_borocode` Int8,
    `pickup_ct2010` String,
    `pickup_boroct2010` String,
    `pickup_cdeligibil` String,
    `pickup_ntacode` FixedString(4),
    `pickup_ntaname` String,
    `pickup_puma` UInt16,
    `dropoff_nyct2010_gid` UInt8,
    `dropoff_ctlabel` Float32,
    `dropoff_borocode` UInt8,
    `dropoff_ct2010` String,
    `dropoff_boroct2010` String,
    `dropoff_cdeligibil` String,
    `dropoff_ntacode` FixedString(4),
    `dropoff_ntaname` String,
    `dropoff_puma` UInt16
")
'''
)

[]

In [65]:
client.execute('SHOW TABLES')

[('trips',), ('trips_2',)]

In [66]:
client.execute("SELECT count() FROM trips")

[(5998971,)]

In [67]:
tip_amount = client.execute("SELECT round(avg(tip_amount), 2) FROM trips")  
print(tip_amount)

[(1.68,)]


In [68]:
type(tip_amount)

list

In [69]:
tip_amount[0][0]

1.68

In [70]:
avg_price = client.execute(
'''
SELECT
    passenger_count,
    ceil(avg(total_amount),2) AS average_total_amount
FROM trips
GROUP BY passenger_count
'''
)

In [71]:
import pandas as pd

In [72]:
df = pd.DataFrame(avg_price, columns=("passenger_count", "average_total_amount"))

In [73]:
df

,passenger_count,average_total_amount
0,0,22.69
1,1,15.97
2,2,17.15
3,3,16.76
4,4,17.33
5,5,16.35
6,6,16.04
7,7,59.80
8,8,36.41
9,9,9.81


In [74]:
pickup_num = client.execute(
'''
SELECT
    pickup_date,
    pickup_ntaname,
    SUM(1) AS number_of_trips
FROM trips
GROUP BY pickup_date, pickup_ntaname
ORDER BY pickup_date ASC
'''
)

In [75]:
df = pd.DataFrame(pickup_num, columns=("pickup_date", "pickup_ntaname", "number_of_trips"))
df

,pickup_date,pickup_ntaname,number_of_trips
0,2015-07-01,Brooklyn Heights-Cobble Hill,39
1,2015-07-01,Old Astoria,15
2,2015-07-01,Flushing,3
3,2015-07-01,Yorkville,1134
4,2015-07-01,Gramercy,1032
...,...,...,...
8746,2015-09-30,Hollis,3
8747,2015-09-30,Claremont-Bathgate,3
8748,2015-09-30,Turtle Bay-East Midtown,3180
8749,2015-09-30,East New York (Pennsylvania Ave),3


In [76]:
trip_minutes = client.execute(
'''
SELECT
    avg(tip_amount) AS avg_tip,
    avg(fare_amount) AS avg_fare,
    avg(passenger_count) AS avg_passenger,
    count() AS count,
    truncate(date_diff('second', pickup_datetime, dropoff_datetime)/3600) as trip_minutes
FROM trips
WHERE trip_minutes > 0
GROUP BY trip_minutes
ORDER BY trip_minutes DESC
'''
)

In [77]:
df = pd.DataFrame(trip_minutes, columns=("avg_tip", "avg_fare", "avg_passenger", "count", "trip_minutes"))
df

,avg_tip,avg_fare,avg_passenger,count,trip_minutes
0,0.980000,10.000000,1.500000,6,458.0
1,1.182368,14.493378,2.060201,4485,23.0
2,2.115957,23.228723,2.468085,141,22.0
3,1.121818,13.681818,1.909091,33,21.0
4,0.321818,18.045455,2.363636,33,20.0
5,2.149000,17.550000,1.500000,30,19.0
6,4.537059,37.000000,1.764706,51,18.0
7,1.425000,30.388462,1.423077,78,17.0
8,1.770500,20.050000,2.200000,60,16.0
9,1.490870,22.782609,2.130435,69,15.0
